# Toronto and it's Neighborhoods
## An exercise in K-means Clustering & Segmenting

### Importing required libraries

In [2]:
!conda install -c anaconda beautifulsoup4 --yes
import requests
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np

Solving environment: done


==> WARNING: A newer version of conda exists. <==
  current version: 4.5.11
  latest version: 4.7.12

Please update conda by running

    $ conda update -n base -c defaults conda



## Package Plan ##

  environment location: /home/jupyterlab/conda/envs/python

  added / updated specs: 
    - beautifulsoup4


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    numpy-base-1.15.4          |   py36h81de0dd_0         4.2 MB  anaconda
    numpy-1.15.4               |   py36h1d66e8a_0          35 KB  anaconda
    beautifulsoup4-4.8.1       |           py36_0         153 KB  anaconda
    openssl-1.1.1              |       h7b6447c_0         5.0 MB  anaconda
    soupsieve-1.9.5            |           py36_0          61 KB  anaconda
    mkl_fft-1.0.6              |   py36h7dd41cf_0         150 KB  anaconda
    certifi-2019.11.28         |           py36_0         156 KB  a

### Scraping and sorting data from Wiki Page to a Pandas DataFrame

In [34]:
#getting relevant page content
wiki_page = requests.get("https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M")
soup = BeautifulSoup(wiki_page.content, 'html.parser')

#creating DF
table = soup.find('tbody')
rows = table.select('tr')
row = [r.get_text() for r in rows]
df = pd.DataFrame(row)


#cleanling and organizing DF
df1 = df[0].str.split('\n', expand=True)
df1.rename(columns=df1.iloc[0], inplace=True)
df1.drop(df1.index[0], inplace=True)
df1.head()

Not assigned       78
Runnymede           2
St. James Town      2
North Midtown       1
Wexford             1
                   ..
Wilson Heights      1
Sullivan            1
Cliffcrest          1
Milliken            1
Summerhill East     1
Name: Neighbourhood, Length: 208, dtype: int64

### Ignore cells with no Borough & Combine Neighborhoods with same Borough

In [32]:
# new df with no NAN Boroughs
df2 = df1[df1.Borough != 'Not assigned']

#new df with combined Neighborhoods
df3 = df2.groupby(['Postcode', 'Borough'], sort = False).agg(','.join)
df3.reset_index(inplace=True)
df3.head()

,Postcode,Borough,Neighbourhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,Harbourfront
3,M6A,North York,"Lawrence Heights,Lawrence Manor"
4,M7A,Queen's Park,Not assigned


### Unassigned Neighborhoods become same as Borough (Only Queen's Park has this problem)

In [35]:
df3.replace("Not assigned", "Queen's Park", inplace=True)
df3.head()

,Postcode,Borough,Neighbourhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,Harbourfront
3,M6A,North York,"Lawrence Heights,Lawrence Manor"
4,M7A,Queen's Park,Queen's Park


In [36]:
df3.shape

(103, 3)